In [ ]:
import pynmea2
import folium
import pandas as pd

In [ ]:
def read_nmea2(lines,deci=1):
    data = []
    for line in lines:
        try:
            msg = pynmea2.parse(line)
            
            if isinstance(msg, pynmea2.GGA):
                lat = msg.latitude
                lon = msg.longitude
                timestamp = msg.timestamp
                elevation = msg.altitude
                
                data.append({'Latitude': lat, 'Longitude': lon, 'Timestamp': timestamp, 'Elevation': elevation})
                
        except pynmea2.ParseError:
            pass
        
    return pd.DataFrame(data)[::deci]


def create_folium_gps_map_folium(gps_data):
    
    # calculate center 
    center_lat = gps_data['Latitude'].mean()
    center_lon = gps_data['Longitude'].mean()
    
    city_map = folium.Map(location=[center_lat, center_lon], zoom_start=12)
    
    # make a PolyLine to represent the driving path 
    path = folium.PolyLine(
        locations=gps_data[['Latitude','Longitude']].values,
        color='blue',
        weight=5,
        opacity=0.7
    )
    path.add_to(city_map)
    
    return city_map

In [ ]:
nmea_file_path = './data/NMEA_GPS_20231002.txt'
deci = 60 #decimate for mapping

# open NMEA text file
with open(nmea_file_path, 'r') as file:
    lines = file.readlines()
    gps_df = read_nmea2(lines,deci=deci)

# get map for the GPS data
city_map = create_folium_gps_map_folium(gps_df)

# display 
city_map